<a href="https://colab.research.google.com/github/ericbonelli/Cientista-de-Dados_EBAC/blob/main/Mod13_Tarefa01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda

Vamos trabalhar com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [28]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import statsmodels.api as sm
import patsy

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
df_renda = pd.read_csv('/content/drive/MyDrive/Especialização em IA/Cientista de Dados - Ebac/Desenvolvimento de aplicações com Python/Regressão 1/previsao_de_renda.csv')
df_renda.head()

,Unnamed: 0,data_ref,id_cliente,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,0,2015-01-01,15056,F,False,True,0,Empresário,Secundário,Solteiro,Casa,26,6.602740,1.0,8060.34
1,1,2015-01-01,9968,M,True,True,0,Assalariado,Superior completo,Casado,Casa,28,7.183562,2.0,1852.15
2,2,2015-01-01,4312,F,True,True,0,Empresário,Superior completo,Casado,Casa,35,0.838356,2.0,2253.89
3,3,2015-01-01,10639,F,False,True,1,Servidor público,Superior completo,Casado,Casa,30,4.846575,3.0,6600.77
4,4,2015-01-01,7064,M,True,False,0,Assalariado,Secundário,Solteiro,Governamental,33,4.293151,1.0,6475.97


In [13]:
#Verificar dados faltantes e nulos
df_renda.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

In [14]:
# eliminar as colunas com dados faltantes
df_renda.dropna(inplace=True)
df_renda.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12427 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             12427 non-null  int64  
 1   data_ref               12427 non-null  object 
 2   id_cliente             12427 non-null  int64  
 3   sexo                   12427 non-null  object 
 4   posse_de_veiculo       12427 non-null  bool   
 5   posse_de_imovel        12427 non-null  bool   
 6   qtd_filhos             12427 non-null  int64  
 7   tipo_renda             12427 non-null  object 
 8   educacao               12427 non-null  object 
 9   estado_civil           12427 non-null  object 
 10  tipo_residencia        12427 non-null  object 
 11  idade                  12427 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  12427 non-null  float64
 14  renda                  12427 non-null  float64
dtypes: bool

In [15]:
#Verificar dados duplicados
df_renda.duplicated().sum()
print(f'A base possui {data_renda.duplicated().sum()} dados duplicados')

A base possui 0 dados duplicados


1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
    - Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.
    - Mantenha sempre a categoria mais frequente como casela de referência
    - Avalie os parâmetros e veja se parecem fazer sentido prático.  


2. Remova a variável menos significante e analise:
    - Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
    - Observe os parâmetros e veja se algum se alterou muito.  


3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor.
    

In [18]:
#1- Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
# Transformar as variáveis em Dummmy
data_cleaned = df_renda.drop(columns=["Unnamed: 0", "data_ref", "id_cliente"])
data_cleaned = pd.get_dummies(data_cleaned, drop_first=True)
data_cleaned.head()

,posse_de_veiculo,posse_de_imovel,qtd_filhos,idade,tempo_emprego,qt_pessoas_residencia,renda,sexo_M,tipo_renda_Bolsista,tipo_renda_Empresário,...,educacao_Superior incompleto,estado_civil_Separado,estado_civil_Solteiro,estado_civil_União,estado_civil_Viúvo,tipo_residencia_Casa,tipo_residencia_Com os pais,tipo_residencia_Comunitário,tipo_residencia_Estúdio,tipo_residencia_Governamental
0,False,True,0,26,6.602740,1.0,8060.34,False,False,True,...,False,False,True,False,False,True,False,False,False,False
1,True,True,0,28,7.183562,2.0,1852.15,True,False,False,...,False,False,False,False,False,True,False,False,False,False
2,True,True,0,35,0.838356,2.0,2253.89,False,False,True,...,False,False,False,False,False,True,False,False,False,False
3,False,True,1,30,4.846575,3.0,6600.77,False,False,False,...,False,False,False,False,False,True,False,False,False,False
4,True,False,0,33,4.293151,1.0,6475.97,True,False,False,...,False,False,True,False,False,False,False,False,False,True


In [21]:
# Ajuste um modelo de regressão linear múltipla para explicar renda (ou uma transformação de renda) de acordo com as demais variáveis.
data_cleaned.columns = data_cleaned.columns.str.replace(r"[^\w]", "_", regex=True)
data_cleaned['log_renda'] = np.log(data_cleaned['renda'])
formula = 'log_renda ~ ' + ' + '.join(data_cleaned.drop(columns=['renda', 'log_renda']).columns)
modelo_c = smf.ols(formula=formula, data=data_cleaned).fit()
print(modelo_c.summary())

                            OLS Regression Results                            
Dep. Variable:              log_renda   R-squared:                       0.357
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     287.5
Date:                Sun, 02 Feb 2025   Prob (F-statistic):               0.00
Time:                        22:07:23   Log-Likelihood:                -13568.
No. Observations:               12427   AIC:                         2.719e+04
Df Residuals:                   12402   BIC:                         2.737e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
                                            coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------

In [24]:
# R2, R2 Adjustado e AIC
print(f'R2: {modelo_c.rsquared}')
print(f'R2 Ajustado: {modelo_c.rsquared_adj}')
print(f'AIC: {modelo_c.aic}')

R2: 0.35747040581738665
R2 Ajustado: 0.3562270007004391
AIC: 27185.298825201382


In [29]:
# 2. Remova a variável menos significante e analise:
y_adj, X_adj = patsy.dmatrices("np.log(renda) ~ idade + tempo_emprego + posse_de_veiculo + posse_de_imovel + sexo_M + tipo_renda_Empresário", data_cleaned, return_type="dataframe")
modelo_adj = sm.OLS(y_adj, X_adj).fit()
print(modelo_adj.summary())

                            OLS Regression Results                            
Dep. Variable:          np.log(renda)   R-squared:                       0.352
Model:                            OLS   Adj. R-squared:                  0.351
Method:                 Least Squares   F-statistic:                     1123.
Date:                Sun, 02 Feb 2025   Prob (F-statistic):               0.00
Time:                        22:13:04   Log-Likelihood:                -13624.
No. Observations:               12427   AIC:                         2.726e+04
Df Residuals:                   12420   BIC:                         2.731e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept     

In [30]:
# R2, R2 Adjustado e AIC
print(f'R2: {modelo_adj.rsquared}')
print(f'R2 Ajustado: {modelo_adj.rsquared_adj}')
print(f'AIC: {modelo_adj.aic}')

R2: 0.3516686137643271
R2 Ajustado: 0.3513554101960973
AIC: 27261.006214958055


In [32]:
# Compare R² values

r2_values = {
    'Modelo Var. Múltiplos': modelo_c.rsquared,
    'Modelo Var. Múltiplos Ajustado': modelo_adj.rsquared
}

print("\nComparação do R²:")
for model_name, r2 in r2_values.items():
    print(f"{model_name}: {r2:.4f}")



Comparação do R²:
Modelo Var. Múltiplos: 0.3575
Modelo Var. Múltiplos Ajustado: 0.3517


In [36]:
# Compare R²Adj values
r2_values = {
    'Modelo Var. Múltiplos': modelo_c.rsquared_adj,
    'Modelo Var. Múltiplos Ajustado': modelo_adj.rsquared_adj
}

print("\nComparação do R²Adj:")
for model_name, r2 in r2_values.items():
    print(f"{model_name}: {r2:.4f}")


Comparação do R²Adj:
Modelo Var. Múltiplos: 0.3562
Modelo Var. Múltiplos Ajustado: 0.3514


In [38]:
# Compare AIC values
aic_values = {
    'Modelo Var. Múltiplos': modelo_c.aic,
    'Modelo Var. Múltiplos Ajustado': modelo_adj.aic
}

print("\nComparação do AIC:")
for model_name, aic in aic_values.items():
    print(f"{model_name}: {aic:.4f}")


Comparação do AIC:
Modelo Var. Múltiplos: 27185.2988
Modelo Var. Múltiplos Ajustado: 27261.0062


**Apesar de termos diminuido a quantidade de variáveis, o primeiro modelo ficou levemente melhor nos 3 indicadores analisados, R2, R2_adj e AIC**